The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

``` bash
mkdir students
if 
wget -O students_file.csv "https://www.dropbox.com/scl/fi/bxv17nrbrl83vw6qrkiu9/LCP_22-23_students.csv?rlkey=47fakvatrtif3q3qw4q97p5b7&dl=1"

if [ -f "./students_file.csv" ]
then  
echo "The file is already present\!"
else
wget -O students_file.csv "https://www.dropbox.com/scl/fi/bxv17nrbrl83vw6qrkiu9/LCP_22-23_students.csv?rlkey=47fakvatrtif3q3qw4q97p5b7&dl=1"
echo " File created \!"
fi
```

Alternative (apparently on my mac "=" for strings means ==):
``` bash

for file in ./*; do if [ "${file}" = "./students_file.csv" ]
then  
echo "The file is already present\!"
else
wget -O students_file.csv "https://www.dropbox.com/scl/fi/bxv17nrbrl83vw6qrkiu9/LCP_22-23_students.csv?rlkey=47fakvatrtif3q3qw4q97p5b7&dl=1"
echo " File created \!"
fi
done
```

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

``` bash
#!/bin/bash
#select a line with grep, and then cut the columns for Surnames and Names
grep "PoD" students_file.csv | cut -f1,2 -d , > students_PoD
grep "Physics" students_file.csv | cut -f1,2 -d , > students_Physics
```

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

``` bash
#!/bin/bash

for letter in {A..Z}; do
#-d: the delimiter is a comma -f1: extract column 1 (Family Name(s))
#  awk 'NR > 1' students_file.csv, skip 1 line: makes sure that the first line (header) is not selected 
# grec -c: count
# >: writes, >>: appends
 count=$(awk 'NR > 1' students_file.csv | cut -d, -f1| grep -c "^$letter")
 echo "$letter $count" >> letter_counts.txt

done
```

1\.d Find out which is the letter with most counts.

``` bash
#!/bin/bash
max_count=0
max=A
for letter in {A..Z}; do
#-d: the delimiter is a comma -f1: extract column 1 (Family Name(s))
#  awk 'NR > 1' students_file.csv, skip 1 line: makes sure that the first line (header) is not selected 
# grec -c: count
# >: writes, >>: appends
 count=$(awk 'NR > 1' students_file.csv | cut -d, -f1| grep -c "^$letter")
 echo "$letter $count" >> letter_counts.txt
  if [ $count -gt $max_count ]; then
    max=$letter
    max_count=$count
  fi
done
 echo $max
```

1\. Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

``` bash
#!/bin/bash
students=$(awk 'NR > 1' students_file.csv | cut -d, -f1,2)
ln=0 # line counter

for student in $students; do
    ln=$((ln + 1)) # Increment line counter

    count=$((ln % 18)) # Calculate modulus
    echo "$student" >> "student_mod$count"
done
```

This version needs to be improved (there are some issues with line handling). Here a less clear, but improved version:

``` bash
#!/bin/bash

ln=0 # line counter

# Read each student name directly from the file, skipping the header line
tail -n +2 students_file.csv | cut -d, -f1,2 | while read -r student; do
    ln=$((ln + 1)) # Increment line counter

    count=$((ln % 18)) # Calculate modulus
    echo "$student" >> "student_mod$count"
done
```

### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

``` bash
#!/bin/bash

#cp data.csv ./data.txt
# tr: it's a sort of substitution. Usage: tr [options] set1 [set2]
#-d: Deletes characters in set1 from the input.
#-s: Squeezes repeating characters in set1 to a single occurrence.

tail -n+5 data.csv | cut -d, -f1-6 | tr -d , > data.txt #select the data lines | cut by "," selecting all columns | delete "," 
```

2\.b How many even numbers are there?

``` bash
#delete the file oddity each time before creating it again
rm oddity.txt

#write each number (not line! Single number) % 2 and write it on line
for xx in $(tail -n+1 data.txt); do
echo $(( xx % 2 )) >> oddity.txt
done

# Count the number of even numbers (0 means even)
even=$(grep -c "0" oddity.txt)

# Print the number of even numbers
echo "The number of even numbers is: $even"
```

To read each line at a single time we would need to do something like:
``` bash
tail -n +1 data.txt | while read -r line; do #reads all the lines (in this case there isn't an header)


done
```

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

``` bash
#clean the numeric result file
rm -f result.txt

# cut -d["delimiter"]: cuts according to "delimiter"
tail -n +1 data.txt | cut -d" " -f1-3 | while read -r field1 field2 field3; do #reads all the lines (in this case there isn't an header)
# Access each field separately (it's a read option)
  if (( field1**2 + field2**2 + field3**2 > 100*100*3/4 )); then
  echo 1 >> result.txt
  else
  echo 0 >> result.txt
  fi
done

echo "The number of positive results is: $(grep -c "1" result.txt)"

echo "The number of negative results is: $(grep -c "0" result.txt)"
```

2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

``` bash
#read variable_name:    reads from keyboard
#-p: display a prompt
read -p "Please select the number of copies: " n

#for ii in {1..$n} creates problem with the operations !

for ii in $(seq 1 $n); do
    # Clean the file (no need to copy it, just clear it if it exists)
    > data_$ii.txt  # Clear or create the file

    # Process each line of data.txt
    tail -n +1 data.txt | cut -d" " -f1-6 | while read -r field1 field2 field3 field4 field5 field6; do

        # Divide each field by $ii
        field1=$(echo "scale=2; $field1 / $ii" | bc -l)
        field2=$(echo "scale=2; $field2 / $ii" | bc -l)
        field3=$(echo "scale=2; $field3 / $ii" | bc -l)
        field4=$(echo "scale=2; $field4 / $ii" | bc -l)
        field5=$(echo "scale=2; $field5 / $ii" | bc -l)
        field6=$(echo "scale=2; $field6 / $ii" | bc -l)

        # Write the transformed values into the file
        echo "$field1 $field2 $field3 $field4 $field5 $field6" >> data_$ii.txt
    done
done



#bc: Bash itself only supports integer arithmetic by default, so bc is necessary for floating-point calculations.
#-l option for floating point arithmetic
#scale set the number of digits represented

```